In [26]:
import os
import sys
import re
import datetime
import traceback
import pytz
import math
import json
import csv
import chardet
from pprint import pprint
import difflib
from wasabi import color
import requests
from ckanapi import RemoteCKAN, NotFound
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [8]:
#__location__ = os.path.realpath(
#    os.path.join(
#        os.getcwd(),
#        os.path.dirname(__file__)
#    )
#)
__location__ = os.path.abspath('')

In [9]:
requests_kwargs = {"verify": False}

In [35]:
def convert_attributes(json_attr):
    # insert attributes
    try:
        attributes = json.loads(json_attr)
    except json.decoder.JSONDecodeError as e:
        print(f"Error: {e}", file=sys.stderr)
        attributes = []
    attrs = []
    for attribute in attributes:
        m = re.match(
            r"^(?P<name>.+?)(\s*\(technisch: (?P<tech_name>.+?)\))?$", attribute[0]
        )
        assert m, f"Could not match name and tech_name from {attribute[0]}"
        attr = {
            "description": attribute[1],
            "name": m["name"],
            "tech_name": m["tech_name"],
        }
        attrs.append(attr)
    return attrs

def get_csv_attributes(urls):
    # use the first URL
    partial_path = os.path.join(__location__, 'partial.csv')
    download_partial_csv(urls[0], partial_path)

    partial_rows = rows_from_csv(partial_path, delimiter=',')
    return list(partial_rows[0].keys())

def download_partial_csv(url, path):
    remote = requests.Session()
    remote.verify = False # do not verify SSL certs

    r = remote.get(url)
    with open(path, 'wb') as f:
        # break after some chunks
        total_chunks = 10
        for chunk in r.iter_content(1024):
            f.write(chunk)

            total_chunks -= 1
            if total_chunks == 0:
                break

def rows_from_csv(input_file, delimiter=';', input_encoding='detect'):
    if input_encoding == 'detect':
        input_encoding = _detect_file_encoding(input_file)

    with open(input_file, 'r', encoding=input_encoding) as f:
        # if a list of delimiters is given, try to sniff the correct one
        if isinstance(delimiter, list):
            dialect = csv.Sniffer().sniff(f.read(), delimiter)
            f.seek(0)
            reader = csv.DictReader(f, dialect=dialect)
        else:
            reader = csv.DictReader(f, delimiter=delimiter)
        rows = [r for r in reader]
    return rows


def _detect_file_encoding(input_file, line_count=20):
    with open(input_file, 'rb') as f:
        # read some lines
         rawdata = b''.join([f.readline() for _ in range(line_count)])
    return chardet.detect(rawdata)['encoding']

def diff_strings(a, b):
    output = []
    matcher = difflib.SequenceMatcher(None, a, b)
    changed = False
    for opcode, a0, a1, b0, b1 in matcher.get_opcodes():
        if opcode == "equal":
            output.append(a[a0:a1])
        elif opcode == "insert":
            changed = True
            output.append(color(b[b0:b1], fg=16, bg="green"))
        elif opcode == "delete":
            changed = True
            output.append(color(a[a0:a1], fg=16, bg="red"))
        elif opcode == "replace":
            changed = True
            output.append(color(b[b0:b1], fg=16, bg="green"))
            output.append(color(a[a0:a1], fg=16, bg="red"))

    if not changed:
        return ""

    return "".join(output)

In [21]:
BASE_URL = os.getenv('CKAN_BASE_URL')
API_KEY = os.getenv('CKAN_API_KEY')
ckan = RemoteCKAN(BASE_URL, apikey=API_KEY)

In [22]:
BASE_URL

'https://data.integ.stadt-zuerich.ch'

In [23]:
# pagination
more_results = True
max_rows = 10
start = 0

rows = []
while(more_results):  
    results = ckan.call_action(
        "package_search", {"q": "res_format:csv -tags:geodaten", "start": start, "rows": max_rows}, requests_kwargs=requests_kwargs
    )
    start += max_rows
    more_results = False
    if start > results['count']:
        more_results = False

    for dataset in results["results"]:
        pprint(dataset)
        ckan_attributes = convert_attributes(dataset['sszFields'])
        csv_attributes = get_csv_attributes([r['url'] for r in dataset['resources'] if r['format'].lower() == 'csv'])
        #pprint(csv_attributes)

        row = {
            "author": dataset["author"],
            "dataset_name": dataset['name'],
            "dataset_title": dataset['title'],
            "ckan_attributes": json.dumps([a['tech_name'] or a['name'] for a in ckan_attributes]),
            "csv_attributes": json.dumps(csv_attributes),
        }
        rows.append(row)
rows

c:\Users\sszods\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'proxy.szh.loc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'author': 'Dienstabteilung Verkehr, Sicherheitsdepartement',
 'author_email': None,
 'creator_user_id': 'd2e1adfa-4bdb-4022-a3b9-6d56ffbb9f31',
 'dataQuality': 'Siehe Bemerkungen',
 'dataType': ['Einzeldaten'],
 'dateFirstPublished': '13.03.2020',
 'dateLastUpdated': '29.11.2022',
 'extras': [{'key': 'harvest_object_id',
             'value': 'b85e5a43-157f-46d4-95d1-ffc2dbbf9df8'},
            {'key': 'harvest_source_id',
             'value': 'cc11cf5d-747f-4a65-a0a2-7786b99e3c76'},
            {'key': 'harvest_source_title',
             'value': 'INT_DWH Dropzone Harvester'}],
 'groups': [{'description': '',
             'display_name': 'Mobilität',
             'id': 'mobilitat',
             'image_display_url': 'https://statistik.stadt-zuerich.ch/Modules/ogd_bspe/group_pics/kategorien-12.png',
             'name': 'mobilitat',
             'title': 'Mobilität'},
            {'description': '',
             'display_name': 'Umwelt',
             'id': 'umwelt',
             'ima

c:\Users\sszods\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'proxy.szh.loc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'author': 'Bevölkerungsamt, Präsidialdepartement',
 'author_email': None,
 'creator_user_id': 'd2e1adfa-4bdb-4022-a3b9-6d56ffbb9f31',
 'dataQuality': 'gut',
 'dataType': ['Datenaggregat'],
 'dateFirstPublished': '08.07.2016',
 'dateLastUpdated': '29.11.2022',
 'extras': [{'key': 'harvest_object_id',
             'value': '4e9f41f7-345d-47c2-ac57-12f486241823'},
            {'key': 'harvest_source_id',
             'value': 'cc11cf5d-747f-4a65-a0a2-7786b99e3c76'},
            {'key': 'harvest_source_title',
             'value': 'INT_DWH Dropzone Harvester'}],
 'groups': [{'description': '',
             'display_name': 'Be\xadvölk\xaderung',
             'id': 'bevolkerung',
             'image_display_url': 'https://statistik.stadt-zuerich.ch/modules/ogd_bspe/group_pics/kategorien-04.png',
             'name': 'bevolkerung',
             'title': 'Be\xadvölk\xaderung'}],
 'id': '0b38d84b-c064-4c90-9d03-dc516bcf51c3',
 'isopen': True,
 'legalInformation': '',
 'license_id': 'cc-zero',

c:\Users\sszods\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'proxy.szh.loc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


...

{'author': 'Bevölkerungsamt, Präsidialdepartement',
 'author_email': None,
 'creator_user_id': 'd2e1adfa-4bdb-4022-a3b9-6d56ffbb9f31',
 'dataQuality': 'siehe Bemerkungen',
 'dataType': ['Datenaggregat'],
 'dateFirstPublished': '21.04.2017',
 'dateLastUpdated': '28.11.2022',
 'extras': [{'key': 'harvest_object_id',
             'value': '34c493a9-64ab-4adc-a53d-a708f084ff05'},
            {'key': 'harvest_source_id',
             'value': 'cc11cf5d-747f-4a65-a0a2-7786b99e3c76'},
            {'key': 'harvest_source_title',
             'value': 'INT_DWH Dropzone Harvester'}],
 'groups': [{'description': '',
             'display_name': 'Be\xadvölk\xaderung',
             'id': 'bevolkerung',
             'image_display_url': 'https://statistik.stadt-zuerich.ch/modules/ogd_bspe/group_pics/kategorien-04.png',
             'name': 'bevolkerung',
             'title': 'Be\xadvölk\xaderung'}],
 'id': '05bf40e4-af57-4d58-abfb-296b28fc337b',
 'isopen': True,
 'legalInformation': '',
 'license_i

c:\Users\sszods\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'proxy.szh.loc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'author': 'Bevölkerungsamt, Präsidialdepartement',
 'author_email': None,
 'creator_user_id': 'd2e1adfa-4bdb-4022-a3b9-6d56ffbb9f31',
 'dataQuality': 'gut',
 'dataType': ['Datenaggregat'],
 'dateFirstPublished': '15.03.2013',
 'dateLastUpdated': '24.11.2022',
 'extras': [{'key': 'harvest_object_id',
             'value': '18f3bfc9-008c-48c7-993f-72ab23fd8c5d'},
            {'key': 'harvest_source_id',
             'value': 'cc11cf5d-747f-4a65-a0a2-7786b99e3c76'},
            {'key': 'harvest_source_title',
             'value': 'INT_DWH Dropzone Harvester'}],
 'groups': [{'description': '',
             'display_name': 'Be\xadvölk\xaderung',
             'id': 'bevolkerung',
             'image_display_url': 'https://statistik.stadt-zuerich.ch/modules/ogd_bspe/group_pics/kategorien-04.png',
             'name': 'bevolkerung',
             'title': 'Be\xadvölk\xaderung'}],
 'id': 'c557b207-00ca-4b2c-8c8b-9cd5bab50557',
 'isopen': True,
 'legalInformation': '',
 'license_id': 'cc-zero',

c:\Users\sszods\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'proxy.szh.loc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'author': 'Dienstabteilung Verkehr, Sicherheitsdepartement',
  'dataset_name': 'int_dwh_sid_dav_verkehrszaehlung_miv_od2031',
  'dataset_title': 'Daten der Verkehrszählung zum motorisierten Individualverkehr (Stundenwerte), seit 2012',
  'ckan_attributes': '["MSID", "MSName", "ZSID", "ZSName", "Achse", "HNr", "Hoehe", "EKoord", "NKoord", "Richtung", "Knummer", "Kname", "AnzDetektoren", "D1ID", "D2ID", "D3ID", "D4ID", "MessungDatZeit", "LieferDat", "AnzFahrzeuge", "AnzFahrzeugeStatus"]',
  'csv_attributes': '["MSID", "MSName", "ZSID", "ZSName", "Achse", "HNr", "Hoehe", "EKoord", "NKoord", "Richtung", "Knummer", "Kname", "AnzDetektoren", "D1ID", "D2ID", "D3ID", "D4ID", "MessungDatZeit", "LieferDat", "AnzFahrzeuge", "AnzFahrzeugeStatus"]'},
 {'author': 'Bevölkerungsamt, Präsidialdepartement',
  'dataset_name': 'int_dwh_bev_monat_bestand_quartier_geschl_ag_herkunft_od3250',
  'dataset_title': 'Bevölkerung nach Monat, Stadtquartier, Geschlecht, Altersgruppe und Herkunft, seit 1998',
  'ck

In [24]:
csv_path = os.path.join(__location__, 'ckan_attributes.csv')

with open(csv_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(
        f,
        list(rows[0].keys()),
        delimiter=',',
        quotechar='"',
        lineterminator='\n',
        quoting=csv.QUOTE_MINIMAL
    )
    writer.writeheader()
    writer.writerows(rows)

In [41]:
with open(csv_path, 'r', newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        csv_attr = json.loads(row['csv_attributes'])
        ckan_attr = json.loads(row['ckan_attributes'])
        csv_attr.sort()
        ckan_attr.sort()

        #ckan_attr.pop()
        #ckan_attr.pop()

        diff = diff_strings(",".join(csv_attr), ",".join(ckan_attr))
        if diff:
            print(diff)
            print(",".join(csv_attr))
            print(",".join(ckan_attr))
        else:
            print("No diff.")

        

        
    

No diff.
No diff.
No diff.
No diff.
No diff.
No diff.
No diff.
Jahr,Quartal,ch_s2,ch_s3,ch_tot,stzh_TZB_Frauen,stzh_TZB_Maenner,stzh_TZB_s2,stzh_TZB_s3,stzh_TZB_tot,stzh_VZA_Frauen,stzh_VZA_Maenner,stzh_VZA_s2,stzh_VZA_s3,stzh_VZA_tot,stzh_VZB_Frauen,stzh_VZB_Maenner,stzh_VZB_s2,stzh_VZB_s3,stzh_VZB_tot,stzh_frau,stzh_mann,stzh_s2,stzh_s3,stzh_tot,zh_s2,zh_s3,zh_tot
Jahr,Quartal,ch_s2,ch_tot,stzh_TZB_Frauen,stzh_TZB_Maenner,stzh_TZB_s2,stzh_TZB_s3,stzh_TZB_tot,stzh_VZA_Frauen,stzh_VZA_Maenner,stzh_VZA_s2,stzh_VZA_s3,stzh_VZA_tot,stzh_VZB_Frauen,stzh_VZB_Maenner,stzh_VZB_s2,stzh_VZB_s3,stzh_VZB_tot,stzh_frau,stzh_mann,stzh_s2,stzh_s3,stzh_tot,zh_s2,zh_s3,zh_tot
Jahr,Quartal,ch_s2,ch_s3,ch_tot,stzh_TZB_Frauen,stzh_TZB_Maenner,stzh_TZB_s2,stzh_TZB_s3,stzh_TZB_tot,stzh_VZA_Frauen,stzh_VZA_Maenner,stzh_VZA_s2,stzh_VZA_s3,stzh_VZA_tot,stzh_VZB_Frauen,stzh_VZB_Maenner,stzh_VZB_s2,stzh_VZB_s3,stzh_VZB_tot,stzh_frau,stzh_mann,stzh_s2,stzh_s3,stzh_tot,zh_s2,zh_s3,zh_tot
No diff.
No diff.


In [ ]:
# download last file from GitHub and compare with today